# Manejo de bases de datos para las ciencias sociales II


## Manipulando la base de datos

- Sorting
- Creación de columnas
- Casting
- Agregando datos
- Manejando valores missing


A continuación veremos cuáles son los métodos más comunes para transformar nuestra base de datos y obtener nueva información.

In [2]:
#importando pandas y leyendo nuestros datos

import pandas as pd


In [3]:
## Leyendo los archivos.
locacion_datos = "aqui  coloca el string con la locación de tus datos"
locacion_datos = "https://www.datosabiertos.gob.pe/sites/default/files/Datos_abiertos_admision_2021_1_2024_1.csv" # este es el mio
df = pd.read_csv(locacion_datos)

In [4]:
df.columns

Index(['IDHASH', 'COLEGIO', 'COLEGIO_DEPA', 'COLEGIO_PROV', 'COLEGIO_DIST',
       'COLEGIO_PAIS', 'COLEGIO_ANIO_EGRESO', 'ESPECIALIDAD', 'ANIO_POSTULA',
       'CICLO_POSTULA', 'DOMICILIO_DEPA', 'DOMICILIO_PROV', 'DOMICILIO_DIST',
       'ANIO_NACIMIENTO', 'NACIMIENTO_PAIS', 'NACIMIENTO_DEPA',
       'NACIMIENTO_PROV', 'NACIMIENTO_DIST', 'SEXO', 'CALIF_FINAL', 'INGRESO',
       'MODALIDAD'],
      dtype='object')

In [ ]:
### Ordenar nuestra base de datos:


In [5]:
## una base de datos en no requiere que esté ordenada, pues la información mostrada es la misma,
# pero a veces es útil para visualizarla o para hacer operaciones.
#df = df.sort_values('ANIO_NACIMIENTO', ascending = False) # por default, ascending es True.
# lo mismo, pero inplace:
#df.sort_values('ANIO_NACIMIENTO', ascending = False, inplace=True)

# ordenar por dos O más columnas:
#df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'])

df = df.sort_values(by=['ANIO_NACIMIENTO', 'NACIMIENTO_PAIS'], ascending=[False, True])



### Creando columnas nuevas.

A continuación veremos varias maneras de crear variables nuevas, dependiendo de las circunstancias de los datos, así como nuestras necesidades/preferencias.



In [7]:
df['COLEGIO2'] = df['COLEGIO']

In [10]:
df['MAYOR_EDAD']  = df['ANIO_NACIMIENTO'] <= 2007

In [ ]:
# La manera más sencilla:
# Crear una nueva columna a partir de la edad del postulante:
df['MAYOR_EDAD'] = df['ANIO_NACIMIENTO'] <= 2006

Los siguentes métodos que utilizamos para filtrar datos también sirven para crear variables:


`.loc `

Podemos crear nuevas variables ubicando en el dataframe los valores que cumplen cierta condición y reemplazando por otras.

In [18]:
df['SEXO_'] = 0
df.loc[
    df['SEXO'] == 'FEMENINO',
    'SEXO_'] = 1


In [19]:
df.loc[df['SEXO'] == 'FEMENINO', 'SEXO_']

,SEXO_
1078,1
10215,1
10538,1
19453,1
22605,1
...,...
29462,1
15169,1
25891,1
19079,1


In [13]:
pd.crosstab(df['SEXO_'],df['SEXO'])

SEXO,FEMENINO,MASCULINO
SEXO_,,
0,0,32419
1,10097,0


In [ ]:
df['SEXO_'] = 0
df.loc[df['SEXO'] == 'FEMENINO', 'SEXO_'] = 1

In [21]:
l_paises = df['NACIMIENTO_PAIS'].unique()
l_paises.sort()

In [ ]:
l_paises.remove("PERU")

Si queremos cambiar varias categorías a la vez, podemos utilizar  `isin()`, `replace` o `where`:

In [23]:
paises = ['BOLIVIA', 'BRAZIL', 'CHILE', 'COLOMBIA', 'EE.UU.',
       'ITALIA', 'JAPÓN', 'VENEZUELA', 'ARGENTINA', 'ECUADOR', 'ESPAÑA',
       'HONDURAS', 'NO ESPECIFICA', 'PARAGUAY', 'FRANCIA', 'SUIZA',
       'URUGUAY', 'GUATEMALA', 'KOREA', 'MÉXICO', 'SIRIA'] ## df['NACIMIENTO_PAIS'].unique()



In [24]:
df['extranjero'] = 0
df.loc[df['NACIMIENTO_PAIS'].isin(paises), 'extranjero'] = 1

In [26]:
df['extranjero'].value_counts(normalize= True)

,proportion
extranjero,
0,0.994567
1,0.005433


In [28]:
dic = {'FEMENINO': 1, 'MASCULINO': 0}

df['OTRO_SEXO'] = df['SEXO']
df.replace(dic, inplace = True)

/tmp/ipython-input-1815370429.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(dic, inplace = True)


In [ ]:
# import numpy as np
# df['extranjero'] = np.where(df['NACIMIENTO_PAIS'].isin(paises), 1, 0)


In [ ]:
df.columns

Utilizando `eval` podemos hacerlo con una sola expresión:


In [ ]:
df['MAYOR_EDAD'] = df.eval('ANIO_NACIMIENTO <= 2006')
df['INGRESANTES_NOLIMA'] = df.eval("INGRESO == 'NO' & DOMICILIO_DEPA != 'LIMA'")



### Una forma más general: Utilizando apply y funciones lambda.

Recuerdan las funciones labmda que vimos en la clase pasada? Estas resultan muy útiles para hacer operaciones en columnas de un dataframe cuando la vectorización no es posible. A continuación veremos algunos ejemplos:

In [34]:
df['Colegio_'] = df['COLEGIO'].apply(lambda x: x.title())

In [36]:
def volver_a_titulo(x):
  '''
   x : str
  '''
  return x.title()

In [37]:
df['Colegio2'] = df['COLEGIO'].apply(lambda x: volver_a_titulo(x))

In [38]:
df[['COLEGIO', 'Colegio_', 'Colegio2']]

,COLEGIO,Colegio_,Colegio2
1078,NEWTON,Newton,Newton
1592,PITAGORAS DE SAMOA,Pitagoras De Samoa,Pitagoras De Samoa
1656,NEWTON,Newton,Newton
2355,LA SALLE SCHOOL,La Salle School,La Salle School
2491,PERUANO JAPONES LA VICTORIA,Peruano Japones La Victoria,Peruano Japones La Victoria
...,...,...,...
37784,UNIVERSIDAD NACIONAL DE INGENIERÍA,Universidad Nacional De Ingeniería,Universidad Nacional De Ingeniería
25049,UNIVERSIDAD NACIONAL DE INGENIERÍA,Universidad Nacional De Ingeniería,Universidad Nacional De Ingeniería
22259,UNIVERSIDAD NACIONAL DE INGENIERÍA,Universidad Nacional De Ingeniería,Universidad Nacional De Ingeniería
30637,1071 ALFONSO UGARTE,1071 Alfonso Ugarte,1071 Alfonso Ugarte


In [44]:
df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: x <= 2006)


In [45]:
def mayor_edad(x):
    if x <= 2006:
        return True
    else:
        return False

df['MAYOR_EDAD_'] = df['ANIO_NACIMIENTO'].apply(lambda x: mayor_edad(x))

También podemos crear variables categóricas en base a variables contínuas con `cut`:

In [41]:
categ_labels = ['cat_1', 'cat_2', 'cat_3', 'cat_4']
categ_bins = [-1, 5, 10, 15, 20.01]

In [46]:
df['cat_calificacion'] = pd.cut(df['CALIF_FINAL'],
                              bins = categ_bins, labels = categ_labels)

In [48]:
df['cat_calificacion'].value_counts(normalize = True)

,proportion
cat_calificacion,
cat_2,0.465029
cat_1,0.262120
cat_3,0.256566
cat_4,0.016285


### Casting: Cambiando tipos de datos:
Hay veces cuando necetaremos cambiar el tipo de dato de una columna.    

- Por ejemplo, cuando el csv fue leido y una columna que debería ser numérica,  es leída como string.  
- Otra situación se da cuando queremos optimizar el uso de memoria de nuestra base de datos, y por ejemplo, queremos reducir el número de decimales que el sistema guarda.  
- Luego, cuando concatenamos bases de datos, los tipos entre uno y otro pueden no coincidir, y necesitaremos cambiarlos para poder hacer operaciones.


In [ ]:
df['ANIO_NACIMIENTO'] = df['ANIO_NACIMIENTO'].astype('str')

# Cuando leemos los archivos, existe un argumento que se llama dtype, que nos permite especificar el tipo de dato de cada columna.


## Agregando datos


Para hacer agregaciones por columnas: ```axis = 0``` ó ```axis = 'index' ```
Para hacer agregaciones por filas: ```axis = 1``` ó ```axis = 'columns' ```

(dato curioso: El axis 0 en realidad se refiere a las filas, pero lo invocamos para obtener datos agregados por columnas porque es como lo mantenemos "fijo". Lo mismo para hacer operaciones por filas).

| Función     | Descripción                                               |
|-------------|-----------------------------------------------------------|
| `mean`      | Calcula el promedio de los valores.                       |
| `median`    | Encuentra la mediana de los valores.                      |
| `sum`       | Suma todos los valores.                                   |
| `min`       | Encuentra el valor mínimo.                                |
| `max`       | Encuentra el valor máximo.                                |
| `quantile`  | Calcula el cuantil especificado de los valores.           |
| `std`       | Calcula la desviación estándar de los valores.            |
| `var`       | Calcula la varianza de los valores.                       |
| `count`     | Cuenta el número de valores no nulos.                     |


In [49]:
pbi = pd.read_csv("https://raw.githubusercontent.com/ccsuehara/python_para_las_ccss/refs/heads/main/Clase%204/data/pbi.csv")

departamentos = ['Amazonas', 'Áncash', 'Apurímac', 'Arequipa', 'Ayacucho',
       'Cajamarca', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín',
       'La Libertad', 'Lambayeque', 'Lima', 'Prov. Const. del Callao',
       'Región Lima', 'Provincia de Lima', 'Loreto', 'Madre de Dios',
       'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes',
       'Ucayali']

In [52]:
pbi['total'] = pbi[departamentos].sum(axis=1)
pbi

,Anio,Amazonas,Áncash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,total
0,2007,1778775,15672771,1824181,16991831,2975676,8159499,10913725,2475279,3200861,...,1864543,7525100,5486459,12651720,5888474,3266254,5143137,1637185,3054659,429428525
1,2008,1930947,16854588,1688564,18885807,3401175,9319769,11663686,2613850,3464132,...,1902177,8663619,5416732,13580502,6328455,3598432,5016741,1902059,3212843,467206837
2,2009,2058318,16400826,1623801,19032479,3750401,10050467,13631820,2696095,3499798,...,2033411,8436303,5040946,13998851,6587873,3740600,4823192,2082047,3243767,471433790
3,2010,2210682,16013215,1765744,20158733,3922514,10140905,15405459,2817536,3739082,...,2229180,8457008,4702403,15106528,6980479,4034361,5338879,2307874,3351315,512037914
4,2011,2287107,16155687,1869417,21038813,4111349,10595497,17384466,2909215,3955589,...,2454999,7785269,4641887,16366999,7384505,4245537,5466509,2168906,3548168,548673683
5,2012,2551601,17666947,2110908,22033542,4482971,11270583,17711332,3143661,4380310,...,1950139,7756800,4880072,17066135,7734458,4752177,5526840,2440755,3882453,581030275
6,2013,2682266,18478843,2342674,22629103,4906299,11086928,20708699,3174927,4642728,...,2240082,8598669,4885819,17746782,8294320,4828116,5781849,2491026,3947464,613934474
7,2014,2824603,16028265,2437434,22773308,4879476,10855588,20723581,3281748,4799787,...,1923155,8371348,5046668,18750443,8487313,5173301,6094038,2607186,3957775,631216479
8,2015,2782128,17584621,2630345,23524592,5162331,10798612,21071852,3265820,5114983,...,2346810,8693747,5211406,18866671,8519702,5466266,6598403,2540855,4163474,652658518
9,2016,2784366,18365696,6343065,29623112,5177917,10581305,21898270,3212948,5345445,...,2663699,8635514,5329324,18924869,9075999,5588107,6506248,2507255,4176645,675963828


In [53]:
pbi[departamentos].mean() # axis = 0 es por default. b

,0
Amazonas,2.547614e+06
Áncash,1.763926e+07
Apurímac,3.588958e+06
Arequipa,2.387133e+07
Ayacucho,4.608745e+06
Cajamarca,1.049615e+07
Cusco,1.818440e+07
Huancavelica,3.076869e+06
Huánuco,4.620494e+06
Ica,1.371483e+07


In [ ]:
## Hallando la suma total de pbi departamental
pbi['total'] = pbi[departamentos].sum(axis=1)

## Hallando el promedio de pbi departamental de los últimos años:

pbi[departamentos].mean() # axis = 0 es por default. b

## Utilizando el groupby:

El groupby nos va a permitir agrupar los datos según una columna y aplicar una función a cada grupo. Es conveniente para cuando querramos obtener datos a un nivel más agregado.

In [ ]:
df.columns

In [54]:
df.groupby('ESPECIALIDAD')['CALIF_FINAL'].mean()

,CALIF_FINAL
ESPECIALIDAD,
ARQUITECTURA,7.020420
CIENCIA DE LA COMPUTACIÓN,8.490926
FÍSICA,8.814689
INGENIERÍA AMBIENTAL,7.293185
INGENIERÍA CIVIL,7.653435
INGENIERÍA DE CIBERSEGURIDAD,8.230595
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL,7.282430
INGENIERÍA DE MINAS,7.485044
INGENIERÍA DE PETRÓLEO Y GAS NATURAL,7.398056


In [60]:
res.reset_index()

Index(['ESPECIALIDAD', 'INGRESO', 'CALIF_FINAL'], dtype='object')

In [ ]:
df.groupby('ESPECIALIDAD')['CALIF_FINAL'].mean()
#.sort_values(ascending = False)

In [56]:
res = df.groupby(['ESPECIALIDAD', 'INGRESO'])['CALIF_FINAL'].mean()
## qué pasa si colocas un .reset_index() al final?


In [62]:
res = df.groupby(['ESPECIALIDAD']).agg(
    {'CALIF_FINAL': ['mean', 'max', 'min'],
     'ANIO_NACIMIENTO': ['mean', 'max', 'min']
    }

)

res.columns

MultiIndex([(    'CALIF_FINAL', 'mean'),
            (    'CALIF_FINAL',  'max'),
            (    'CALIF_FINAL',  'min'),
            ('ANIO_NACIMIENTO', 'mean'),
            ('ANIO_NACIMIENTO',  'max'),
            ('ANIO_NACIMIENTO',  'min')],
           )

###  Una forma generalizada de hacer groupbys:

In [ ]:

## aquí utilizamos el método agg para aplicar varias funciones a la vez.
nueva_agregacion =df.groupby(['ESPECIALIDAD']).agg(
{'CALIF_FINAL': ['mean', 'max','min'],
 'ANIO_NACIMIENTO': ['min', 'median']}
)



In [67]:
## los nombres de las columnas son una lista de strings, que al tener varios niveles, tendrán un multiIndex.
# En este curso no lidiaaremos con  multiIndex, así que te propongo la siguiente solución:
nueva_agregacion = res.copy(deep = True)
nueva_agregacion.columns = ['_'.join(col).strip() for col in nueva_agregacion.columns.values]


['CALIF_FINAL_mean',
 'CALIF_FINAL_max',
 'CALIF_FINAL_min',
 'ANIO_NACIMIENTO_mean',
 'ANIO_NACIMIENTO_max',
 'ANIO_NACIMIENTO_min']

In [ ]:
## Otra forma de hacerlo

new_col = []
for col in nueva_agregacion.columns.values:
    a = ''.join(col).strip()
    new_col.append(a)
nueva_agregacion.columns = new_col


In [ ]:
# Otra forma de hacerlo
# df.groupby(['ESPECIALIDAD', 'INGRESO']).agg(nueva_var =  ('CALIF_FINAL', 'mean')).reset_index()

## resumiendo datos con pivot_table()
Pandas tiene un método que nos permite hacer tablas dinámicas (a lo Excel) para resumir información.



In [68]:
pd.pivot_table(df,
               index = 'ESPECIALIDAD', # filas
               columns = 'INGRESO', # columnas
               values = 'CALIF_FINAL',
               aggfunc = 'mean'
)

INGRESO,NO,SI
ESPECIALIDAD,,
ARQUITECTURA,6.503551,13.034397
CIENCIA DE LA COMPUTACIÓN,7.069774,11.852496
FÍSICA,7.010372,12.053004
INGENIERÍA AMBIENTAL,5.955341,11.664649
INGENIERÍA CIVIL,6.817769,13.721998
INGENIERÍA DE CIBERSEGURIDAD,7.122163,12.833407
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL,5.982545,11.258549
INGENIERÍA DE MINAS,6.249669,11.840015
INGENIERÍA DE PETRÓLEO Y GAS NATURAL,6.188209,12.195724


In [ ]:
pd.pivot_table(df, values = 'CALIF_FINAL', #variable con los valores que vamos a resumir
                index = 'ESPECIALIDAD', # filas
                  columns = 'INGRESO', # columnas
                    aggfunc = 'mean' # función de agregación
                    )

In [69]:
pd.pivot_table(df,
                values = ['CALIF_FINAL', 'ANIO_NACIMIENTO'],
                index = 'ESPECIALIDAD',
                columns = 'INGRESO',
                aggfunc = {'CALIF_FINAL': ['mean', 'median'],
                            'ANIO_NACIMIENTO': 'median'})

ANIO_NACIMIENTO          \
                                                      median           
INGRESO                                                   NO      SI   
ESPECIALIDAD                                                           
ARQUITECTURA                                          2004.0  2003.0   
CIENCIA DE LA COMPUTACIÓN                             2004.0  2003.0   
FÍSICA                                                2004.0  2004.0   
INGENIERÍA AMBIENTAL                                  2004.0  2003.0   
INGENIERÍA CIVIL                                      2004.0  2003.0   
INGENIERÍA DE CIBERSEGURIDAD                          2005.0  2004.0   
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL          2004.0  2002.0   
INGENIERÍA DE MINAS                                   2004.0  2004.0   
INGENIERÍA DE PETRÓLEO Y GAS NATURAL                  2003.0  2002.0   
INGENIERÍA DE SISTEMAS                                2005.0  2004.0   
INGENIERÍA DE SOFTWARE                                2006.0  2005.0   
INGENIERÍA DE TELECOMUNICACIONES                      2004.0  2003.0   
INGENIERÍA ECONÓMICA                                  2004.0  2003.0   
INGENIERÍA ELECTRÓNICA                                2004.0  2003.0   
INGENIERÍA ELÉCTRICA                                  2004.0  2003.0   
INGENIERÍA ESTADÍSTICA                                2005.0  2004.0   
INGENIERÍA FÍSICA                                     2004.0  2003.0   
INGENIERÍA GEOLÓGICA                                  2004.0  2003.0   
INGENIERÍA INDUSTRIAL                                 2004.0  2003.0   
INGENIERÍA MECATRÓNICA                                2004.0  2004.0   
INGENIERÍA MECÁNICA                                   2004.0  2003.0   
INGENIERÍA MECÁNICA-ELÉCTRICA                         2004.0  2003.0   
INGENIERÍA METALÚRGICA                                2004.0  2003.0   
INGENIERÍA NAVAL                                      2004.0  2003.0   
INGENIERÍA PETROQUÍMICA                               2004.0  2003.0   
INGENIERÍA QUÍMICA                                    2004.0  2004.0   
INGENIERÍA SANITARIA                                  2003.0  2002.0   
INGENIERÍA TEXTIL                                     2004.0  2002.0   
MATEMÁTICA                                            2004.0  2004.0   
QUÍMICA                                               2003.0  2003.0   

                                             CALIF_FINAL                     \
                                                    mean             median   
INGRESO                                               NO         SI      NO   
ESPECIALIDAD                                                                  
ARQUITECTURA                                    6.503551  13.034397  6.4105   
CIENCIA DE LA COMPUTACIÓN                       7.069774  11.852496  7.1660   
FÍSICA                                          7.010372  12.053004  7.0950   
INGENIERÍA AMBIENTAL                            5.955341  11.664649  5.9880   
INGENIERÍA CIVIL                                6.817769  13.721998  6.5000   
INGENIERÍA DE CIBERSEGURIDAD                    7.122163  12.833407  7.1085   
INGENIERÍA DE HIGIENE Y SEGURIDAD INDUSTRIAL    5.982545  11.258549  5.9500   
INGENIERÍA DE MINAS                             6.249669  11.840015  6.2220   
INGENIERÍA DE PETRÓLEO Y GAS NATURAL            6.188209  12.195724  5.8880   
INGENIERÍA DE SISTEMAS                          6.309844  13.354662  5.9735   
INGENIERÍA DE SOFTWARE                          6.947527  13.778621  6.8400   
INGENIERÍA DE TELECOMUNICACIONES                7.158423  13.012861  7.3060   
INGENIERÍA ECONÓMICA                            6.381207  11.592202  6.2945   
INGENIERÍA ELECTRÓNICA                          7.217361  13.006515  7.2840   
INGENIERÍA ELÉCTRICA                            6.973840  12.869580  7.0240   
INGENIERÍA ESTADÍSTICA                          6.598870  11.396682  6.4790   
INGENIERÍA FÍSIC

In [70]:
### Formatos Long-Wide:

pbi

,Anio,Amazonas,Áncash,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,total
0,2007,1778775,15672771,1824181,16991831,2975676,8159499,10913725,2475279,3200861,...,1864543,7525100,5486459,12651720,5888474,3266254,5143137,1637185,3054659,429428525
1,2008,1930947,16854588,1688564,18885807,3401175,9319769,11663686,2613850,3464132,...,1902177,8663619,5416732,13580502,6328455,3598432,5016741,1902059,3212843,467206837
2,2009,2058318,16400826,1623801,19032479,3750401,10050467,13631820,2696095,3499798,...,2033411,8436303,5040946,13998851,6587873,3740600,4823192,2082047,3243767,471433790
3,2010,2210682,16013215,1765744,20158733,3922514,10140905,15405459,2817536,3739082,...,2229180,8457008,4702403,15106528,6980479,4034361,5338879,2307874,3351315,512037914
4,2011,2287107,16155687,1869417,21038813,4111349,10595497,17384466,2909215,3955589,...,2454999,7785269,4641887,16366999,7384505,4245537,5466509,2168906,3548168,548673683
5,2012,2551601,17666947,2110908,22033542,4482971,11270583,17711332,3143661,4380310,...,1950139,7756800,4880072,17066135,7734458,4752177,5526840,2440755,3882453,581030275
6,2013,2682266,18478843,2342674,22629103,4906299,11086928,20708699,3174927,4642728,...,2240082,8598669,4885819,17746782,8294320,4828116,5781849,2491026,3947464,613934474
7,2014,2824603,16028265,2437434,22773308,4879476,10855588,20723581,3281748,4799787,...,1923155,8371348,5046668,18750443,8487313,5173301,6094038,2607186,3957775,631216479
8,2015,2782128,17584621,2630345,23524592,5162331,10798612,21071852,3265820,5114983,...,2346810,8693747,5211406,18866671,8519702,5466266,6598403,2540855,4163474,652658518
9,2016,2784366,18365696,6343065,29623112,5177917,10581305,21898270,3212948,5345445,...,2663699,8635514,5329324,18924869,9075999,5588107,6506248,2507255,4176645,675963828


In [72]:
pbi_ = pbi.melt(id_vars="Anio")
 ## En caso tuviesemos más de una variable

In [76]:
pd.pivot_table(pbi_, values = 'value', index = 'Anio', columns = 'variable').reset_index() ##kwargs

variable,Anio,Amazonas,Apurímac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,Huánuco,Ica,...,Prov. Const. del Callao,Provincia de Lima,Puno,Región Lima,San Martín,Tacna,Tumbes,Ucayali,total,Áncash
0,2007,1778775.0,1824181.0,16991831.0,2975676.0,8159499.0,10913725.0,2475279.0,3200861.0,8793956.0,...,13386423.0,111447422.0,5888474.0,11404858.0,3266254.0,5143137.0,1637185.0,3054659.0,429428525.0,15672771.0
1,2008,1930947.0,1688564.0,18885807.0,3401175.0,9319769.0,11663686.0,2613850.0,3464132.0,10415637.0,...,14627416.0,121299582.0,6328455.0,12488983.0,3598432.0,5016741.0,1902059.0,3212843.0,467206837.0,16854588.0
2,2009,2058318.0,1623801.0,19032479.0,3750401.0,10050467.0,13631820.0,2696095.0,3499798.0,10841974.0,...,14288888.0,123008998.0,6587873.0,11612252.0,3740600.0,4823192.0,2082047.0,3243767.0,471433790.0,16400826.0
3,2010,2210682.0,1765744.0,20158733.0,3922514.0,10140905.0,15405459.0,2817536.0,3739082.0,11607992.0,...,15689336.0,136634487.0,6980479.0,12300019.0,4034361.0,5338879.0,2307874.0,3351315.0,512037914.0,16013215.0
4,2011,2287107.0,1869417.0,21038813.0,4111349.0,10595497.0,17384466.0,2909215.0,3955589.0,12883432.0,...,17563803.0,147616320.0,7384505.0,13562753.0,4245537.0,5466509.0,2168906.0,3548168.0,548673683.0,16155687.0
5,2012,2551601.0,2110908.0,22033542.0,4482971.0,11270583.0,17711332.0,3143661.0,4380310.0,13067505.0,...,18070083.0,157630141.0,7734458.0,13897097.0,4752177.0,5526840.0,2440755.0,3882453.0,581030275.0,17666947.0
6,2013,2682266.0,2342674.0,22629103.0,4906299.0,11086928.0,20708699.0,3174927.0,4642728.0,14394675.0,...,19196199.0,166311098.0,8294320.0,14893394.0,4828116.0,5781849.0,2491026.0,3947464.0,613934474.0,18478843.0
7,2014,2824603.0,2437434.0,22773308.0,4879476.0,10855588.0,20723581.0,3281748.0,4799787.0,14809397.0,...,19828477.0,172725823.0,8487313.0,15468191.0,5173301.0,6094038.0,2607186.0,3957775.0,631216479.0,16028265.0
8,2015,2782128.0,2630345.0,23524592.0,5162331.0,10798612.0,21071852.0,3265820.0,5114983.0,15295581.0,...,20209052.0,178267707.0,8519702.0,15992567.0,5466266.0,6598403.0,2540855.0,4163474.0,652658518.0,17584621.0
9,2016,2784366.0,6343065.0,29623112.0,5177917.0,10581305.0,21898270.0,3212948.0,5345445.0,15325191.0,...,20690862.0,183403718.0,9075999.0,16146749.0,5588107.0,6506248.0,2507255.0,4176645.0,675963828.0,18365696.0


In [73]:
pbi_

,Anio,variable,value
0,2007,Amazonas,1778775
1,2008,Amazonas,1930947
2,2009,Amazonas,2058318
3,2010,Amazonas,2210682
4,2011,Amazonas,2287107
...,...,...,...
359,2015,total,652658518
360,2016,total,675963828
361,2017,total,692450688
362,2018,total,720802642


In [ ]:
df_wide = pd.pivot_table(df, values = 'CALIF_FINAL', index = 'ESPECIALIDAD', columns = 'INGRESO', aggfunc = 'mean').reset_index()
df_wide.melt(id_vars = 'ESPECIALIDAD', value_vars = ['NO', 'SI'])

### Manejo de datos faltantes:
Los datos faltantes pueden suceder por varias razones:
-  Cuando no se necesita registrar los datos de dicha observación (por ejemplo, nivel educativo para menores de 3 años).  
-  Sin embargo, esto puede afectar nuestro análisis si son errores en el levantamiento de la información.    
    - ejemplo de variables sensibles de levantamiento.
    - errores de levantamiento de datos.
    
Su identifcación y tratamiento es importante.


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42516 entries, 1078 to 35616
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   IDHASH               42516 non-null  object  
 1   COLEGIO              42516 non-null  object  
 2   COLEGIO_DEPA         42290 non-null  object  
 3   COLEGIO_PROV         42288 non-null  object  
 4   COLEGIO_DIST         42288 non-null  object  
 5   COLEGIO_PAIS         18942 non-null  object  
 6   COLEGIO_ANIO_EGRESO  42516 non-null  int64   
 7   ESPECIALIDAD         42461 non-null  object  
 8   ANIO_POSTULA         42516 non-null  int64   
 9   CICLO_POSTULA        42516 non-null  int64   
 10  DOMICILIO_DEPA       42516 non-null  object  
 11  DOMICILIO_PROV       42516 non-null  object  
 12  DOMICILIO_DIST       42516 non-null  object  
 13  ANIO_NACIMIENTO      42516 non-null  int64   
 14  NACIMIENTO_PAIS      42516 non-null  object  
 15  NACIMIENTO_DEPA      

In [78]:
## IDentificando datos faltantes:
df.isna().sum() # nos da la cantidad de datos faltantes por columna.

,0
IDHASH,0
COLEGIO,0
COLEGIO_DEPA,226
COLEGIO_PROV,228
COLEGIO_DIST,228
COLEGIO_PAIS,23574
COLEGIO_ANIO_EGRESO,0
ESPECIALIDAD,55
ANIO_POSTULA,0
CICLO_POSTULA,0


In [ ]:
## Observando datos faltantes de un subconjunto de columnas:
df[['COLEGIO_DEPA', 'NACIMIENTO_DEPA']].isna().sum()

df = df.fillna(0) # reemplaza los datos faltantes por 0.


df['CALIF_FINAL'].fillna(df['CALIF_FINAL'].mean())
#
df[['COLEGIO_DEPA', 'NACIMIENTO_DEPA']].fillna('No especificado')

# Otros métodos: notna(), dropna()


In [82]:
df['CALIF_FINAL_imputado'] =  df['CALIF_FINAL'].fillna(df['CALIF_FINAL'].mean())


In [79]:
df[df['COLEGIO_PAIS'].isna()]

,IDHASH,COLEGIO,COLEGIO_DEPA,COLEGIO_PROV,COLEGIO_DIST,COLEGIO_PAIS,COLEGIO_ANIO_EGRESO,ESPECIALIDAD,ANIO_POSTULA,CICLO_POSTULA,...,MODALIDAD,COLEGIO2,MAYOR_EDAD,SEXO_,extranjero,OTRO_SEXO,Colegio_,Colegio2,MAYOR_EDAD_,cat_calificacion
1078,8B41F491D388F43391AF74CA99E19152CE5DD42E4FDCF7...,NEWTON,JUNIN,SATIPO,SATIPO,NaN,2025,NaN,2024,1,...,INTERESADO,NEWTON,False,1,0,1,Newton,Newton,False,cat_2
1592,A95FEC2C4B78CDA09630CAC45ADFFDE017B1ADEBF393B0...,PITAGORAS DE SAMOA,HUANCAVELICA,HUANCAVELICA,HUANCAVELICA,NaN,2024,NaN,2024,1,...,INTERESADO,PITAGORAS DE SAMOA,False,0,0,0,Pitagoras De Samoa,Pitagoras De Samoa,False,cat_3
1656,F4F0E786B14B052D3DFAC91234A58CF675904C30EC4E09...,NEWTON,JUNIN,SATIPO,SATIPO,NaN,2025,NaN,2024,1,...,INTERESADO,NEWTON,False,0,0,0,Newton,Newton,False,cat_2
2355,7E83DF3B67D6AF48CB80796C4352AB40E175C486B3DD61...,LA SALLE SCHOOL,HUANUCO,HUANUCO,HUANUCO,NaN,2024,NaN,2024,1,...,INTERESADO,LA SALLE SCHOOL,False,0,0,0,La Salle School,La Salle School,False,cat_2
2491,782B538013DCEC09A9F2AFCF88BCE6449D7B297E71E56E...,PERUANO JAPONES LA VICTORIA,LIMA,LIMA,MAGDALENA DEL MAR,NaN,2024,ARQUITECTURA,2023,2,...,ORDINARIO,PERUANO JAPONES LA VICTORIA,False,0,0,0,Peruano Japones La Victoria,Peruano Japones La Victoria,False,cat_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28618,626D8D4C517B9E6A12FD3EEC67B91F7C80C5AE69033BC8...,105 PEDRO CORONADO ARRASCUE,LIMA,LIMA,LIMA,NaN,1980,INGENIERÍA INDUSTRIAL,2023,1,...,ORDINARIO,105 PEDRO CORONADO ARRASCUE,True,0,0,0,105 Pedro Coronado Arrascue,105 Pedro Coronado Arrascue,True,cat_1
11900,2E61EAEDAEC22CF315224BD1B14CEC680D45EDC2027C37...,1071 ALFONSO UGARTE,LIMA,LIMA,SAN ISIDRO,NaN,1978,INGENIERÍA ECONÓMICA,2023,2,...,ORDINARIO,1071 ALFONSO UGARTE,True,0,0,0,1071 Alfonso Ugarte,1071 Alfonso Ugarte,True,cat_1
31555,2E61EAEDAEC22CF315224BD1B14CEC680D45EDC2027C37...,CEBA - 1071 ALFONSO UGARTE,LIMA,LIMA,SAN ISIDRO,NaN,1978,INGENIERÍA ECONÓMICA,2023,1,...,ORDINARIO,CEBA - 1071 ALFONSO UGARTE,True,0,0,0,Ceba - 1071 Alfonso Ugarte,Ceba - 1071 Alfonso Ugarte,True,cat_2
22259,E73E0ED0C93FD4C2DA409C342F7B0BE0843885E360E55B...,UNIVERSIDAD NACIONAL DE INGENIERÍA,NaN,NaN,NaN,NaN,1974,INGENIERÍA DE SISTEMAS,2024,1,...,EXTRAORDINARIO - TITULADOS O GRADUADOS UNI,UNIVERSIDAD NACIONAL DE INGENIERÍA,True,0,0,0,Universidad Nacional De Ingeniería,Universidad Nacional De Ingeniería,True,cat_1
